In [1]:
import numpy as np
import pandas as pd
import datetime

additional_path = '/tf/Lecture/Covid_19_Prediction_Stacking/data/owid-covid-data.csv'
D_path = '/tf/Lecture/Covid_19_Prediction_Stacking/data/time_series_covid19_confirmed_global.csv'
A_path = '/tf/Lecture/Covid_19_Prediction_Stacking/data/time_series_covid19_deaths_global.csv'
R_path = '/tf/Lecture/Covid_19_Prediction_Stacking/data/time_series_covid19_recovered_global.csv'

In [12]:
hard_hit_country = ['US', 'India', 'Brazil', 'Russia', 'France', 'Italy']
relapse_country = ['Italy', 'Germany', 'Denmark', 'Switzerland', 'Russia', 'Austria']
lack_data_countries = ['Brazil', 'Germany']
# interested_countries = list((set(hard_hit_country) | set(relapse_country)) - set(lack_data_countries))
interested_countries = list(set(relapse_country) - set(lack_data_countries))
all_column_name = 'iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index'

column_names = ['iso_code',
                'continent',
                'location',
                'date',
                'total_cases',
                'new_cases',
                'new_cases_smoothed',
                'total_deaths',
                'new_deaths',
                'new_deaths_smoothed',
                'total_cases_per_million',
                'new_cases_per_million',
                'new_cases_smoothed_per_million',
                'total_deaths_per_million',
                'new_deaths_per_million',
                'new_deaths_smoothed_per_million',
                'icu_patients',
                'icu_patients_per_million',
                'hosp_patients',
                'hosp_patients_per_million',
                'weekly_icu_admissions',
                'weekly_icu_admissions_per_million',
                'weekly_hosp_admissions',
                'weekly_hosp_admissions_per_million',
                'total_tests',
                'new_tests',
                'total_tests_per_thousand',
                'new_tests_per_thousand',
                'new_tests_smoothed',
                'new_tests_smoothed_per_thousand',
                'tests_per_case',
                'positive_rate',

                'tests_units',
                'stringency_index',
                'population',
                'population_density',
                'median_age',
                'aged_65_older',
                'aged_70_older',
                'gdp_per_capita',
                'extreme_poverty',
                'cardiovasc_death_rate',
                'diabetes_prevalence',
                'female_smokers',
                'male_smokers',
                'handwashing_facilities',
                'hospital_beds_per_thousand',
                'life_expectancy',
                'human_development_index']

column_names = all_column_name.split(',')


good_features = [
                 # 'stringency_index',
                 'population',
                 'population_density',
                 'median_age',
                 'aged_65_older',
                 'aged_70_older',
                 'gdp_per_capita',
                 # 'extreme_poverty',
                 'cardiovasc_death_rate',
                 'diabetes_prevalence',
                 'male_smokers',
                 'female_smokers',
                 'hospital_beds_per_thousand',
                 'life_expectancy',
                 'human_development_index']
good_time_sequences_features = ['new_tests',                                             'new_tests_per_thousand',
    'total_tests', 'total_tests_per_thousand','positive_rate']
good_generated_features=[
                                         'total_cases_per_million',
                                         'new_cases_per_million',
                                         'new_cases_smoothed_per_million',
                                         'total_deaths_per_million',
                                         'new_deaths_per_million',
                                         'new_deaths_smoothed_per_million',
                                         'total_tests',
                                         'new_tests',
                                         'total_tests_per_thousand',
                                         'new_tests_per_thousand',
                                         'new_tests_smoothed',
                                         'new_tests_smoothed_per_thousand',
                                         'tests_per_case']

input_features=[
    'date',
    # 'start_days',
    # 'date_days'
                                             'total_cases',
    # 'total_cases_smoothed',
    # 'total_cases_per_million',

                                             'new_cases',
                                             # 'new_cases_smoothed',
                                             # 'new_cases_per_million',
                                             # 'new_cases_smoothed_per_million',
                                             'total_deaths',
                                             'new_deaths',
                                             # 'new_deaths_smoothed',
                                             # 'new_deaths_per_million',
                                             # 'new_deaths_smoothed_per_million',
    'total_recovered',
                                             'new_recovered',
                                             # 'new_recovered_per_million',
                                             'total_tests',
                                             'new_tests',
                                             # 'new_tests_per_thousand',
                                             # 'total_tests_per_thousand',
    'death_rate',
    'recovery_rate',
    'positive_rate']


In [13]:
addition_data = pd.read_csv(additional_path)
addition_data['location'].replace('United States', 'US', inplace=True)
addition_data = addition_data[addition_data['location'].isin(interested_countries)].reset_index(drop=True)
addition_data['datetime'] = 1
def generate_date_time(row):
    date_str = row['date'].split('-')
    year = int(date_str[0])
    month = int(date_str[1])
    day = int(date_str[2])
    date_time = int(datetime.date(year, month, day).strftime('%j'))
    if year > 2020:
        date_time += 366
    row['datetime'] = date_time
    row['date'] = '{}/{}/{}'.format(month, day, year - 2000)
    return row
addition_data = addition_data.apply(generate_date_time, axis=1)


In [14]:
addition_seq_df = addition_data[['location', 'date'] + good_time_sequences_features]
addition_seq_df = addition_seq_df[addition_data['total_cases']>0].reset_index(drop=True)


In [15]:
# for feature in good_time_sequences_features:
feature = 'new_tests'
print('----------------')
print('==' + feature + '==')
for country in interested_countries:
    print(country)
    t_data = addition_seq_df[addition_seq_df['location']==country][feature]
    # print(t_data.isnull().value_counts())
    # print(t_data.value_counts())
    t_filled_data = t_data.fillna(method='ffill').fillna(t_data.min())
    # print(t_filled_data.value_counts())
    # for i, v in enumerate(t_filled_data):
    #     print(i, v)

    for i in t_filled_data.index.values:
        addition_seq_df.at[i,feature] = t_filled_data[i]

    # t_data = addition_seq_df[addition_seq_df['location']==country][feature]
    # print(t_data.isnull().value_counts())




----------------
==new_tests==
Switzerland
Italy
Russia
Denmark
Austria


In [16]:
# feature = 'tests_per_case'
# feature_df = interested_data[['location', 'date', feature]]
# feature_df = feature_df.reset_index()
# feature_with_data = feature_df[feature_df[feature].isnull()==False]
# feature_with_data['location'].value_counts()

In [17]:
D_data = pd.read_csv(D_path)
D_interested = D_data[(D_data['Country/Region'].isin(interested_countries)) & (D_data['Province/State'].isnull())]
D_interested = D_interested.reset_index(drop=True).drop(columns=['Province/State', 'Lat', 'Long'])
D_interested = D_interested.rename(columns={'Country/Region':'location'})
D_interested = D_interested.set_index('location')
D_interested = D_interested.transpose()

A_data = pd.read_csv(A_path)
A_interested = A_data[(A_data['Country/Region'].isin(interested_countries)) & (A_data['Province/State'].isnull())]
A_interested = A_interested.reset_index(drop=True).drop(columns=['Province/State', 'Lat', 'Long'])
A_interested = A_interested.rename(columns={'Country/Region':'location'})
A_interested = A_interested.set_index('location')
A_interested = A_interested.transpose()

R_data = pd.read_csv(R_path)
R_interested = R_data[(R_data['Country/Region'].isin(interested_countries)) & (R_data['Province/State'].isnull())]
R_interested = R_interested.reset_index(drop=True).drop(columns=['Province/State', 'Lat', 'Long'])
R_interested = R_interested.rename(columns={'Country/Region':'location'})
R_interested = R_interested.set_index('location')
R_interested = R_interested.transpose()

In [18]:
demography_data = addition_data[['location'] + good_features].drop_duplicates().reset_index(drop=True).set_index('location')
demography_data['first_discovery_date'] = '1/22/20'
# country = 'France'
# D_interested[country][D_interested[country] > 0.0].index[0]
for country in interested_countries:
    first_D_data = D_interested[country][D_interested[country] > 0.0].index[0]
    demography_data.at[country, 'first_discovery_date'] = first_D_data
demography_data['first_discovery_date']


location
Austria        2/25/20
Denmark        2/27/20
Italy          1/31/20
Russia         1/31/20
Switzerland    2/25/20
Name: first_discovery_date, dtype: object

In [25]:
countries_data = {}
for country in interested_countries:
    discovery_data = demography_data['first_discovery_date'][country]
    days = len(D_interested[country][discovery_data:])
    df = pd.DataFrame(np.zeros(shape=(days, len(input_features))), columns=input_features)
    
    df['date'] = D_interested[country][discovery_data:].index.values
    df['total_cases'] = D_interested[country][discovery_data:].values
    df['total_deaths'] = A_interested[country][discovery_data:].values
    df['total_recovered'] = R_interested[country][discovery_data:].values
    t_addition = addition_seq_df[addition_seq_df['location']==country]
    for i in range(days):
        df.at[i, 'new_tests'] = t_addition[t_addition['date'] == df['date'][i]]['new_tests']

    df.at[0, 'new_cases'] = df['total_cases'][0]
    df.at[0, 'new_deaths'] = df['total_deaths'][0]
    df.at[0, 'new_recovered'] = df['total_recovered'][0]
    df.at[0, 'total_tests'] = df['new_tests'][0]
    
    for i in range(1, days):
        df.at[i, 'new_cases'] = df['total_cases'][i] - df['total_cases'][i - 1]
        df.at[i, 'new_deaths'] = df['total_deaths'][i] - df['total_deaths'][i - 1]
        df.at[i, 'new_recovered'] = df['total_recovered'][i] - df['total_recovered'][i - 1]
        df.at[i, 'total_tests'] = df['total_tests'][i - 1] + df['new_tests'][i]

    df['death_rate'] = df['new_deaths'] / df['total_cases']
    df['recovery_rate'] = df['new_recovered'] / df['total_cases']
    df['positive_rate'] = df['new_cases'] / df['new_tests']

    countries_data[country] = df